In [9]:
import numpy as np
import random
import math
import hashlib
import requests

In [10]:
class BBS:
    def __init__(self, p, q, state = 0):
        self.n = p*q
        self.state = state

        if state == 0:
            self.state = random.randint(2, self.n - 1)
    
    def generate_bytes(self, n: int):
        seq = np.zeros(n, dtype=object)
        seq[0] = self.state

        for i in range(1, n):
            seq[i] = pow(seq[i - 1], 2, self.n)

        self.state = seq[-1]
        seq = np.array(seq % (2**8), dtype=np.uint8) 

        return seq
    
def bytes_to_num(byte_seq):
    res = 0
    for b in byte_seq:
        res = res*(2**8) + int(b)

    return res


In [11]:
OPTIMUS_PRIMES = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 
                  61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 
                  137, 139, 149, 151, 157]
R = {}

for d in OPTIMUS_PRIMES:
    R[d] = [1]
    while R[d].count(R[d][-1]) < 2:
        R[d].append((R[d][-1] * 2) % d)
    R[d].pop()

# ### Метод пробних ділень
def petod_drobnyx_mylen(num):
    b = bin(num)[:1:-1]
    
    if b[0] == '0':
        return 2
    
    for d in OPTIMUS_PRIMES[1::]:
        sum = 0
        for i in range(len(b)):
            sum += int(b[i]) * R[d][i % len(R[d])]
            sum %= d
        
        if sum == 0:
            return d

    return 1
    
# ### Ймовірнісний алгоритм Міллера-Рабіна та загальний алгоритм для знаходження простих чисел
def miller_rabin(num, base):
    i = 1
    while (num - 1) % (2 ** i) == 0:
        i += 1

    k = i - 1
    d = (num - 1) // (2 ** k)

    a_d = pow(base, d, num)

    if a_d == 1:
        return True
    
    a_d2i = a_d
    for j in range(k):
        if a_d2i == (num - 1):
            return True
        
        a_d2i = (a_d2i ** 2) % num

    return False


def check_prime(num, error_prob = 0.001):
    if petod_drobnyx_mylen(num) != 1:
        return False

    t = int(math.ceil(math.log(1 / error_prob, 4)))
    s = 0
    for _ in range(t):
        a = random.randrange(3, num + 1)
        s += int(miller_rabin(num, a))

    return s > (t / 2)

def generate_prime(len: int, excl = []):
    gen = BBS(int('425D2B9BFDB25B9CF6C416CC6E37B59C1F', 16), int('D5BBB96D30086EC484EBA3D7F9CAEB07', 16))

    while True:
        p = bytes_to_num(gen.generate_bytes(len // 8))
        if check_prime(p) and (p not in excl):
            return p

    
def generate_safe_prime(len: int, excl = []):
    gen = BBS(int('425D2B9BFDB25B9CF6C416CC6E37B59C1F', 16), int('D5BBB96D30086EC484EBA3D7F9CAEB07', 16))

    while True:
        seq = gen.generate_bytes(len // 8)
        if seq[0] < 128:
            continue
        
        p = bytes_to_num(seq)
        if not check_prime(p) or (p in excl):
            continue

        q = (p - 1) // 2
        if check_prime(q):
            return p


In [12]:
KEY_LENGTH = 256


class Server:
    __base_url = 'http://asymcryptwebservice.appspot.com/rsa/'
    s = requests.Session()
    n = None
    e = None
    
    # Setup server private key and receive server pub key
    def set_server_key(self, key_l: int) -> (str, str):
        req = f'{self.__base_url}serverKey?keySize={key_l}'
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code} for request {req}")
        r = r.json()
        print(r)
        self.n, self.e = (int(r['modulus'], 16), int(r['publicExponent'], 16))
        return (self.n, self.e)
    
    # Ask server to encrypt
    def encrypt(self, M: str, rec_n, rec_e, type='TEXT'):
        req = f'{self.__base_url}encrypt?modulus={format(rec_n, "X")}&publicExponent={format(rec_e, "X")}&message={M}&type={type}'
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code} for request {req}")
        r = r.json()
        print(r)
        return r['cipherText']
    
    # Ask server to decrypt this message with his private keys
    def decrypt(self, C: str, type='TEXT'):
        req = f'{self.__base_url}decrypt?message={C}&expectedType={type}'
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code} for request {req}")
        r = r.json()
        print(r)
        return r['message']
    
    # Ask server to sign this message with his private keys
    def sign(self, M: str, type='TEXT'):
        req = f'{self.__base_url}sign?message={M}&type={type}'
        r = self.s.get(req)
        # if r.status_code != 200:
        #     raise RuntimeError(f"Incorrect server status code {r.status_code} for request {req}")
        r = r.json()
        print(r)
        return r['signature']
    
    # Verify the message using this public key
    def verify(self, M: str, sign: str, u_n, u_e, type='TEXT'):
        req = f'{self.__base_url}verify?message={M}&type={type}&signature={sign}&modulus={format(u_n, "X")}&publicExponent={format(u_e, "X")}'
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code} for request {req}")
        r = r.json()
        print(r)
        return r['verified']
    
    # Receive a pair (64bit encrypted key, signature for this key) from the server
    def sendKey(self, rec_n, rec_e) -> (str, str):
        req = f'{self.__base_url}sendKey?modulus={format(rec_n, "X")}&publicExponent={format(rec_e, "X")}'
        print(req)
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code} for request {req}")
        r = r.json()
        print(r)
        return (r["key"], r['signature'])
    
    # Ask server to decrypt and verify key encrypted with user's modulo and publicExponent
    def receiveKey(self, K_enc: str, sign, u_n, u_e):
        req = f'{self.__base_url}receiveKey?key={K_enc}&signature={sign}&modulus={format(u_n, "X")}&publicExponent={format(u_e, "X")}'
        r = self.s.get(req)
        if r.status_code != 200:
            raise RuntimeError(f"Incorrect server status code {r.status_code} for request {req}")
        r = r.json()
        print(r)
        return r['verified']


In [17]:
def str2hex(s: str):
    res = ""

    for c in s:
        cb = hex(ord(c))
        res += cb[2::]

    return res

def num2str(n: int):
    text = str()
    while n != 0:
        text += chr(n % 256)
        n //= 256
    
    return text[::-1]


class User:
    def __init__(self, p, q, e = 2**16 + 1, serv = Server()):
        if not check_prime(p) or not check_prime(q):
            raise RuntimeError("p or q is not a prime number.")

        if math.gcd(e, (p-1)*(q-1)) != 1:
            raise RuntimeError("e is not invertible modulo phi(n)")
        
        self.serv = serv
        self.p = p
        self.q = q
        self.e = e
        self.n = p*q
        self.d = pow(e, -1, (self.p - 1)*(self.q - 1))
        self.get_server_public_key(KEY_LENGTH * 2)
    
    def get_server_public_key(self, len: int):
        self.serv.set_server_key(len)


    def send_message(self, M: str):
        s_n = self.serv.n
        s_e = self.serv.e

        if int(str2hex(M), 16) > self.serv.n:
            raise RuntimeError("Cannot send the message. Its' length is larger than server's modulo.")

        C = pow(int(str2hex(M), 16), s_e, s_n)
        
        M1 = self.serv.decrypt(C)

        check = (M1 == M)

    def send_message_sign(self, M: str):
        if self.n > self.serv.n:
            raise RuntimeError("Cannot send the sign. User's modulo is larger than server's modulo.")
        
        S = pow(int(str2hex(M), 16), self.d, self.n)

        auth_check = self.serv.verify(M, S, self.n, self.e)
    
    def receive_message(self, M: str):
        if self.n < int(str2hex(M), 16):
            raise RuntimeError("Cannot receive the message. User's modulo is smaller than message length")
        
        C = self.serv.encrypt(M, self.n, self.e)
        M1 = num2str(pow(int(C, 16), self.d, self.n))

        check = (M1 == M)
        print(check)
        print(M1)
        print(M)
        
    def receive_message_sign(self, M: str):
        S = self.serv.sign(M)
        M1 = num2str(pow(int(S, 16), self.serv.e, self.serv.n))
        
        auth_check = (M1 == M)
        print(auth_check)
        print(M1)
        print(M)
        
        
    def receive_secret_key(self):
        if self.n < self.serv.n:
            raise RuntimeError("Cannot receive the key. User's modulo is smaller than server's.")
        
        s_K, s_S = self.serv.sendKey(self.n, self.e)
        K = pow(int(s_K, 16), self.d, self.n)
        S = pow(int(s_S, 16), self.d, self.n)

        auth_check = (pow(S, self.serv.e, self.serv.n) == K)
        print(auth_check)
        

    def send_secret_key(self, K: str):
        if self.n > self.serv.n:
            raise RuntimeError("Cannot send key. User's modulo is larger than server's.")

        if int(K, 16) > self.serv.n:
            raise RuntimeError("Cannot send the key. Key length is larger than server's modulo.")

        EK = pow(int(K, 16), self.serv.e, self.serv.n)
        ES = pow(pow(int(K, 16), self.d, self.n), self.serv.e, self.serv.n)

        auth_check = self.serv.receiveKey(format(EK, "X"), format(ES, "X"), self.n, self.e)
        print(auth_check)


In [14]:
p1 = generate_safe_prime(KEY_LENGTH - 64)
print(p1)
q1 = generate_safe_prime(KEY_LENGTH - 64, [p1])
print(q1)

print()

p2 = generate_safe_prime(KEY_LENGTH + 64)
print(p2)

q2 = generate_safe_prime(KEY_LENGTH + 64, [p2])
print(q2)

4993526698886872659064256247184334076305121782901632904863
4588211384182887734079715434066451875355374388503903352763

1132909072644384510259557297782274290484337684865104819205045889503350186470795397388372898195467
1768089154458202226947313985523481569499780299316851585901190938252418649721405522885980495424243


In [18]:
u_s = User(p1, q1)
u_r = User(p2, q2)

u_r.receive_secret_key()
u_s.send_secret_key("dfb0cd9586cf2d9ffff".capitalize())
# u.receive_message("Pipapapo")
#u.receive_message_sign("Pipa")


{'modulus': '9948093BF24B0F3C60CFC56257ECD99EA3C45F83F3381E6A58FECA3F414643424312241152178DD028AFB3592CEAA4B20FF03ACC61FC50E6E71F578DC4D2BCC9', 'publicExponent': '10001'}
{'modulus': '9948093BF24B0F3C60CFC56257ECD99EA3C45F83F3381E6A58FECA3F414643424312241152178DD028AFB3592CEAA4B20FF03ACC61FC50E6E71F578DC4D2BCC9', 'publicExponent': '10001'}
http://asymcryptwebservice.appspot.com/rsa/sendKey?modulus=7064C94CF481CDF64D9138129A9840DA770EA6447842F97CB55FB5A92AD72F3E53A8CA1A40BBC2C01F30C3B89980733BAB82A2202DCECB8E3B27A146A78B0D8B8290F44B1DCC0FA65EF85B6717785071&publicExponent=10001
{'key': '122FCD8EF9DD1F4162756CDFFF9972F4A044B0376F2174179F13E52250CB9F71FC4CDEBA98FA4D893EF11B36104433B4AF988987752DC8F925ABBBFD89EE68D23CF52C4D61946AE67333E132E86F22F8', 'signature': '3CF98723C8FECC5D3382765695A52555742F2AA403F120F771F6AA61C4EF5B22B09DF808A59EBAC40BD1D3F67C6EB4247BDADC2BCA07CEAC7E1ECE77DF71FFB26C4992D0C4C141BB398D25CFEE433AAC'}
True
{'key': '0DFB0CD9586CF2D9FFFF', 'verified': True}
True
